In [ ]:
import pandas as pd
import numpy as np
import random
from IPython.display import Image
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import networkx as nx
from networkx.algorithms import community
# Node2Vec
from sklearn.preprocessing import normalize
import math
from tqdm import tqdm
from sklearn.metrics import confusion_matrix


In [ ]:
#utility function to get removed edges
def removedEdges(first,second):
    removed = 0
    for edge in first:
        if edge not in second:
            removed+=1
    return removed
#utility function to get added adges
def addedEdges(first,second):
    added = 0
    for edge in second:
        if edge not in first:
            added+=1
    return added

In [ ]:
def positiveScoreMatBuild(_graph):
    pred_adamic = list(nx.adamic_adar_index(_graph))
    positive_score_mat = np.zeros((70,70))
    for item in pred_adamic:
        positive_score_mat[item[0]][item[1]] = item[2]
    return positive_score_mat

In [ ]:
def _target(mat):
    target = []
    for i in range(len(mat)):
        for j in range(i,len(mat[0])):
            target.append(mat[i][j])
    return target

In [ ]:
add_Edges = []
remove_Edges = []
def batchApproach(first,second,batch_size=10):
    #loading matrix of first state
    first_mat = np.loadtxt(first, dtype=int)
    #removing self loops
    for i in range(0,70):
            first_mat[i][i] = 0
    #building graphs
    G_First = nx.from_numpy_matrix(first_mat)
    #print("Number Of Edges: ",len(G_First.edges()))
    #print("Number Of Nodes: ",len(G_First.nodes()))
    #loading matrix of second state
    second_mat = np.loadtxt(second, dtype=int)
    #removing self loops
    for i in range(0,70):
        second_mat[i][i] = 0
    #building graph of second state
    G_second = nx.from_numpy_matrix(second_mat)
    #G_lMCI = nx.from_numpy_matrix(lmci_mat)
    #print("Number Of Edges: ",len(G_second.edges()))
    #print("Number Of Nodes: ",len(G_second.nodes()))
    #print("Number Of Removed Edges: ",removedEdges(G_First.edges(),G_second.edges()))
    #print("Number of new Edges:",addedEdges(G_First.edges(),G_second.edges()))
    #print(second_mat)
    #print(type(second_mat))
    first_pred = second_mat.copy()
    #prepare the positive score matrix
    positive_score_mat = positiveScoreMatBuild(G_First)
    #positive_score_mat = normalize(positive_score_mat, axis=1, norm='l1')
    #preparing negative score matrix
    G_First_complement = nx.complement(G_First)
    negative_score_mat = positiveScoreMatBuild(G_First_complement)
    #negative_score_mat = normalize(negative_score_mat, axis=1, norm='l1')
    #print(negative_score_mat)
    
    A = addedEdges(G_First.edges(),G_second.edges())
    B = removedEdges(G_First.edges(),G_second.edges())
    removed_edges = B
    added_edges = A
    score=(A+B)*2
    laps = math.ceil((A+B)/batch_size)
    done = []
    for m in range(laps):
        positive_score_mat = positiveScoreMatBuild(G_First)
        G_First_complement = nx.complement(G_First)
        negative_score_mat = positiveScoreMatBuild(G_First_complement)

        for n in range(batch_size):
            if(A>0 and B>0):
                maxIndex = 70
                TopRank=-1
                x=0
                y=0
                for i in range(maxIndex-1):
                    for j in range(i+1,maxIndex):
                        if(first_mat[i][j]==0 and (i,j) not in done):
                            #print(i,j)
                            tempRank = positive_score_mat[i][j]
                            if(tempRank>TopRank):
                                x=i
                                y=j
                                TopRank = tempRank
                #positive_score_mat[x][y] = -positive_score_mat[x][y]
                if(positive_score_mat[x][y]>negative_score_mat[x][y]):
                    #adding link
                    add_Edges.append((x,y,positive_score_mat[x][y],negative_score_mat[x][y]))
                    first_pred[x][y] = 1
                    A-=1
                    done.append((x,y))
                    G_First.add_edge(x,y)
                elif(first_pred[x][y]==1):
                    #removing edge
                    remove_Edges.append((x,y,positive_score_mat[x][y],negative_score_mat[x][y]))
                    first_pred[x][y] = 0
                    B-=1
                    #print(x,y)
                    done.append((x,y))
                    G_First.remove_edge(x, y)
                else:
                    done.append((x,y))
    
        
    
            if(A>0):
                maxIndex = 70
                TopRank=-1
                x=0
                y=0
                for i in range(maxIndex-1):
                    for j in range(i+1,maxIndex):
                        if(first_mat[i][j]==0 and (i,j) not in done):
                            tempRank = positive_score_mat[i][j]
                            if(tempRank>TopRank):
                                x=i
                                y=j
                                TopRank = tempRank
        
                first_pred[x][y] = 1
                A-=1
                done.append((x,y))
                add_Edges.append((x,y,positive_score_mat[x][y],negative_score_mat[x][y]))
                G_First.add_edge(x,y)
                
        
                #print(x,y)
            if(B>0):
                maxIndex = 70
                TopRank=-1
                x=0
                y=0
                for i in range(maxIndex-1):
                    for j in range(i+1,maxIndex):
                        if(first_mat[i][j]==1 and (i,j) not in done):
                            tempRank = negative_score_mat[i][j]
                            if(tempRank>TopRank):
                                x=i
                                y=j
                                TopRank = tempRank
        
                first_pred[x][y] = 0
                remove_Edges.append((x,y,positive_score_mat[x][y],negative_score_mat[x][y]))
                G_First.remove_edge(x, y)
                B-=1
                done.append((x,y))
    diff = np.subtract(second_mat,first_pred)    
    dist = 0
    removed_pred = 0
    added_pred = 0
    for i in range(70):
        for j in range(i,70):
            if diff[i][j]!=0:
                dist+=1
            if diff[i][j]==-1:
                removed_pred+=1
            if diff[i][j]==1:
                added_pred+=1
    #print("score: ",score)
    #removal accuracy
    
    matching  = (score-dist)/score
    removed_accuracy = removed_pred/(2*removed_edges)
    added_accuracy = added_pred/(2*added_edges)
    actual_ = _target(second_mat)
    predicted_ = _target(first_pred)
    print("removed Matching Score:: ",removed_accuracy)
    print("added Matching Score:: ",added_accuracy)
    print("confusion matrix",confusion_matrix(actual_,predicted_))
    print("Matching score:",matching)
    
    #print("Matching Score:: ",matching)
    #print(len(done))
    return matching

In [ ]:
add_Edges = []
remove_Edges = []
accuracy = []
x = [i for i in range(1,20)]
for b in tqdm(range(1,20)):
    acc = batchApproach('../Normal.txt','../eMCI.txt',batch_size=b)
    accuracy.append(acc)
plt.plot(x,accuracy)
plt.show()

In [ ]:
add_Edges = []
remove_Edges = []
accuracy = []
x = [i for i in range(1,20)]
for b in tqdm(range(1,20)):
    acc = batchApproach('../eMCI.txt','../lMCI.txt',batch_size=b)
    accuracy.append(acc)
plt.plot(x,accuracy)
plt.show()

In [ ]:
add_Edges = []
remove_Edges = []
accuracy = []
x = [i for i in range(1,20)]
for b in tqdm(range(1,20)):
    acc = batchApproach('../lMCI.txt','../AD.txt',batch_size=b)
    accuracy.append(acc)
plt.plot(x,accuracy)
plt.show()

In [ ]:
add_Edges = []
remove_Edges = []
accuracy = []
x = [i for i in range(1,20)]
for b in tqdm(range(1,20)):
    acc = batchApproach('../normal.txt','../AD.txt',batch_size=b)
    accuracy.append(acc)
plt.plot(x,accuracy)
plt.show()